In [ ]:
DEBUG = True
N_DEBUG = 6

In [ ]:
N_PHASE_1_3 = 16
N_PHASE_2_4 = 32

Modified experiment of Hase et al. (see simulatability.ipynb)

In [ ]:
import pandas as pd
import os
import numpy as np


In [ ]:
from IPython.core.display import HTML


In [ ]:
from explainer_wrappers import LIME_Explainer, SHAP_Explainer

In [ ]:
from detector_guo import DetectorGuo
from detector_dummy import DetectorDummy

In [ ]:
test = pd.read_pickle("./dataset_train.pkl")
test = test 

In [ ]:
sample = test.sample(n=N_PHASE_1_3+N_PHASE_2_4, random_state=2202)
phase_1_3 = sample[0:N_PHASE_1_3]
phase_2_4 = sample[N_PHASE_1_3:N_PHASE_1_3+N_PHASE_2_4]

In [ ]:
documents_phase_1_3 = phase_1_3["answer"]
gold_labels_phase_1_3 = phase_1_3["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py

documents_phase_2_4 = phase_2_4["answer"]
gold_labels_phase_2_4 = phase_2_4["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py

In [ ]:
detector = DetectorGuo()#( machine_watermark="Alcohol")

In [ ]:
explainer = LIME_Explainer(detector)

# Phase 1
Predictions and GT but no explanations

In [ ]:
prompt_template_phase_1 = """
<p><b>This is a {kind_of_document} document.</b></p>
<p>The detector {correctly_or_wrongly} predicted that this document was... </p>
<p>&emsp; ... machine generated with {p_machine} % confidence.</p>
<p>&emsp; ... human written with {p_human} % confidence.</p> 


<div style="float:left;">{document}</div>
"""

In [ ]:
for gold_label, document in zip(gold_labels_phase_1_3, documents_phase_1_3):
    p_machine, p_human = detector.predict_proba([document])[0]

    display(HTML(prompt_template_phase_1.format(
    p_machine=int(p_machine*100), 
    p_human=int(p_human*100),
    kind_of_document= "machine generated" if gold_label == False else "human written", 
    correctly_or_wrongly= "correctly" if detector.predict_label([document])[0] == gold_label else "wrongly", 
    document = document
    )))

# Phase 2
Predict model output without explanations

In [ ]:
prompt_template_phase_2 = """
Please try to guess the decition of the detector:<br />
<div style="float:left;">{document}</div>
<br /> <br />
<p>MACHINE | HUMAN</p>
"""

In [ ]:
for document in documents_phase_2_4:
   
    display(HTML(prompt_template_phase_2.format(
    document = document
    )))

# Phase 3:
Phase 1 + explanations

In [ ]:
prompt_template_phase_3 = """
<p><b>This is a {kind_of_document} document.</b></p>
<p>The detector {correctly_or_wrongly} predicted that this document was... </p>
<p>&emsp; ... machine generated with {p_machine} % confidence.</p>
<p>&emsp; ... human written with {p_human} % confidence.</p> 
<div style="float:left; height:30em;">{barplot_machine}{barplot_human}</div>


<div style="float:left;">{highlighted_text}</div>
"""

In [ ]:
expa = None

In [ ]:
for gold_label, document in zip(gold_labels_phase_1_3, documents_phase_1_3):
  #  explainer.delete_cached_explanation(document)
    # display(HTML(explainer.get_vanilla_visualization_HTML(document)))
    p_machine, p_human = detector.predict_proba([document])[0]
    machine, human = explainer.get_barplots_HTML(document)
    display(HTML(prompt_template_phase_3.format(
    p_machine=int(p_machine*100), 
    p_human=int(p_human*100),
    barplot_machine=machine,
    barplot_human=human,
    kind_of_document= "machine generated" if gold_label == False else "human written", 
    correctly_or_wrongly= "correctly" if detector.predict_label([document])[0] == gold_label else "wrongly", 
    highlighted_text=explainer.get_highlighted_text_HTML(document),
    )))

In [ ]:
from lime.explanation import id_generator
from sklearn.utils import check_random_state
import json
random_state = 42

In [ ]:
explainer = SHAP_Explainer(detector)

In [ ]:
HTML(explainer.get_visualization_HTML(document))

In [ ]:
s = "Alcohol is a diuretic , which means that it causes excessive urination , also the big amounts of fluid you are consuming contributes to more urine formation . Excessive urination leads to loss of ions such as sodium , potassium and magnesium , but the amount of water lost in urine is more than the amount of ions lost ( urine is less concentrated an looks clear ) . That creates an increased concentration of ions in blood . To counter this , the cells push the water inside them to the blood , therefore decreasing the amount of water in them , and this causes dehydration and feeling of thirst , which is a symptom of being hangover ."

In [ ]:
"diuretic" in s